# Workshop 08

## Realizzazione di Villa Pisani di Bagnolo

![villa_pisani](../images/workshop08/VillaPisani_1.jpg)
![villa_pisani](../images/workshop08/VillaPisani_2.jpg)

* Definizione di funzioni **parametriche**
    * Per consentire libertà di utilizzo (variando i parametri)
    * Per favorire il riuso in altri contesti (realizzazione di altri tipi di edifici)

In [1]:
from larlib import *
from numpy import *

Evaluating fenvs.py..
...fenvs.py imported in 0.005525 seconds


* Texture e colori

In [2]:
texture_pietra = 'texture/sand3.png'
texture_pietraP0 = 'texture/mattoni3.png'
texture_lawn = 'texture/prato.png'
texture_brecciolino = 'texture/brecciolino2.png'
texture_tetto = 'texture/tetto3.png'
texture_pavimentoP1 = 'texture/marmo.png'
texture_pavimentoP2 = 'texture/marmo.png'
texture_pareti = 'texture/marmo-bianco.png'
texture_finestre = 'texture/marmo.png'
texture_persiane = 'texture/legno1.jpg'

* Misure di riferimento

In [3]:
wall_thickness = 0.3

window_panel_size = 8

first_floor_height = 14

facet_thickness = 2

roofs_height = 1.5

front_staircase_height = 2
front_staircase_radius = 5.9

lawn_size_x = 80
lawn_size_y = 100

side_windows_number = 4
rear_windows_number = 6

In [4]:
front_villa_size = window_panel_size*1.0*rear_windows_number
side_villa_size = window_panel_size*1.0*side_windows_number

blank_space = ((window_panel_size-2)/2.0)-2

second_floor_X = (lawn_size_x-(side_villa_size+2*blank_space))/2.0
second_floor_Y = (lawn_size_y-(front_villa_size+2*blank_space))/2.0

facet_Y = (((lawn_size_x-(side_villa_size+2.0*blank_space))/2.0)+side_villa_size+blank_space*2.0)-facet_thickness
facet_Z = 3 + 0.3 + wall_thickness*3
facet_X = second_floor_Y + window_panel_size + blank_space*2.0

floor_height_0 = 3 + 0.5 + wall_thickness*3.0

second_floor_height = 3 + 0.3 + wall_thickness*4 + first_floor_height + 0.1

roof_height = second_floor_height + first_floor_height*0.4+wall_thickness
roof_X = second_floor_X+side_villa_size-window_panel_size
roof_Y = second_floor_Y

walls_height = second_floor_height + first_floor_height*0.6

* Mappa cilindrica

In [5]:
def cylMap():
    x = lambda k: k[1] * COS(k[0])
    y = lambda k: k[1] * SIN(k[0])
    z = lambda k: k[2]
    return MAP([x,y,z])

* Funzioni di supporto

In [6]:
def getMapSemiarco(r,altezza):
    x = lambda p : r*sin(p[0])*cos(p[0])
    y = lambda p : r*cos(p[0])*cos(p[0])
    z = lambda p : altezza
    dom = INTERVALS(PI/2)(63)

    map = MAP([x,y,z])(dom)
    return map

def getMap(r,altezza):
    x = lambda p : r*sin(p[0])*cos(p[0])
    y = lambda p : r*cos(p[0])*cos(p[0])
    z = lambda p : altezza
    dom = INTERVALS(PI)(32)

    map = MAP([x,y,z])(dom)
    
    return map

def Cubo_Angoli_Smussati(r,altezza,fattore_smussamento):
    fattore_smussamento = fattore_smussamento * r
    elemento = OFFSET([fattore_smussamento,fattore_smussamento,fattore_smussamento])(JOIN([getMap(r,0),getMap(r,altezza)]))
    
    return elemento

def getlawn(x,y):
    lawn = TEXTURE(texture_lawn)(CUBOID([x,y,0.1]))
    brecciolino = TEXTURE(texture_brecciolino)(T(2)(lawn_size_y/2.0-((window_panel_size*2.0-blank_space*2.0)/2.0))(CUBOID([second_floor_Y,window_panel_size*2.0-blank_space*2.0,0.15])))
    base = STRUCT([lawn, brecciolino])
    return base

lawn = getlawn(lawn_size_x,lawn_size_y)

def getknoll(altezza,lunghezza_totale, lunghezza_cima,spessore):
    side = (lunghezza_totale-lunghezza_cima)*1.0/2.0
    
    V1 = [[0,0],[side,altezza],[side+lunghezza_cima,altezza],[lunghezza_totale,0]]
    FV = [[0,1],[1,2],[2,3],[3,0]]
   
    hill = JOIN(STRUCT(MKPOLS((V1,FV))))
    hill = T(2)(-(lunghezza_totale/2.0))(R([1,2])(PI/2)(R([2,3])(PI/2)(MULTEXTRUDE(hill)(spessore))))
    
    return TEXTURE(texture_lawn)(hill)

knoll = T([1,2])([facet_Y+facet_thickness,lawn_size_y/2.0])(getknoll(2.5,40,14,second_floor_X))

lawn_hill = TEXTURE(texture_lawn)(STRUCT([lawn,knoll]))

### Porte e finestre

* Realizzazione finestre

In [7]:
def hex_material(color, light, trasparence):

    def hex_to_rgb(value):
        value = value.lstrip('#')
        lv = len(value)
        return map(lambda x: x/255., list(int(value[i:i + lv // 3], 16) for i in range(0, lv, lv // 3)))

    params = hex_to_rgb(color) + [.1] + \
             hex_to_rgb(light) + [trasparence] +\
             hex_to_rgb(light) + [.1] +\
             hex_to_rgb("#000000") + [.1] +\
             [100]

    return MATERIAL(params)

def getVetro(altezza,larghezza,spessore):
    vetro = CUBOID([altezza,spessore,larghezza])
    vetro = hex_material("#ffffff", "#ffffff", .4)(vetro)
    return vetro

* Finestra piano terra

In [8]:
def getMattoniLaterali():
    
    MattoneFinestraPiccolo = CUBOID([0.5,0.2,0.3])
    MattoneFinestraGrande = CUBOID([0.8,0.2,0.3])
    getMattoniLaterali = STRUCT([MattoneFinestraPiccolo,
                              T(3)(-0.35)(MattoneFinestraGrande),
                              T(3)(-0.35*2)(MattoneFinestraPiccolo),
                              T(3)(-0.35*3)(MattoneFinestraGrande),
                              T(3)(-0.35*4)(MattoneFinestraPiccolo)
                             ])
    return TEXTURE(texture_pietraP0)(getMattoniLaterali)

def getMattoniLaterali2():
    
    MattoneFinestraPiccolo = CUBOID([0.5,0.2,0.3])
    MattoneFinestraGrande = CUBOID([0.8,0.2,0.3])
    getMattoniLaterali = STRUCT([T(3)(0.35)(MattoneFinestraGrande),
                              MattoneFinestraPiccolo,
                              T(3)(-0.35)(MattoneFinestraGrande),
                              T(3)(-0.35*2)(MattoneFinestraPiccolo),
                              T(3)(-0.35*3)(MattoneFinestraGrande),
                              T(3)(-0.35*4)(MattoneFinestraPiccolo),
                              T(3)(-0.35*5)(MattoneFinestraGrande)
                             ])
    return getMattoniLaterali

def getFinestrella(lunghezza_pannello_finestra_p0, numero_pannelli):

    Finestrella1 = STRUCT([
        INSR(PROD)([QUOTE([0,2,-2,(lunghezza_pannello_finestra_p0-2)/2.0]), QUOTE([0,wall_thickness]), QUOTE([3])]),
        INSR(PROD)([QUOTE([-2,2]), QUOTE([0,wall_thickness]), QUOTE([0.6,-1.8,0.6])])
        ])
    
    DimensioneSpazioBianco = ((lunghezza_pannello_finestra_p0-2)/2.0)-2
    SpazioBianco = CUBOID([DimensioneSpazioBianco,wall_thickness,3])
    
    Finestrella = LEFT([Finestrella1,SpazioBianco])
    
    RinghieraFinestrella = T([1,2,3])([2,wall_thickness/2,0.5])(COMP([STRUCT, NN(10)])([COLOR(GRAY)(Cubo_Angoli_Smussati(0.04,2,1)), T(1)(0.2)]))

    MattoniLaterali = getMattoniLaterali()

    MattoneFinestraPiccolo = CUBOID([0.5,0.2,0.3])
    MattoneFinestraGrande = CUBOID([0.8,0.2,0.3])
   
    MattoniFinestra = TEXTURE(texture_pietraP0)((T([1,2,3])([4,wall_thickness,2])(STRUCT([T([1,3])([0.05,0.05])(MattoniLaterali),
                                                              T([1,3])([-2.05,-1.05])(R([1,3])(PI)(MattoniLaterali)),
                                                              T([1,3])([-0.5,-0.35*5])(MattoneFinestraGrande),
                                                              T([1,3])([-1.35,-0.35*5])(MattoneFinestraGrande),
                                                              T([1,3])([-2.25,-0.35*5])(MattoneFinestraGrande),
                                                              T([1,3])([-0.5,0.45])(MattoneFinestraGrande),
                                                              T([1,3])([-1.35,0.45])(MattoneFinestraGrande),
                                                              T([1,3])([-2.25,0.45])(MattoneFinestraGrande)
                                                             ]))))
    
    FinestrellaCompleta = T(1)(((lunghezza_pannello_finestra_p0-2)/2.0)-2)(STRUCT([Finestrella, RinghieraFinestrella, MattoniFinestra]))
    
    blank_space = ((lunghezza_pannello_finestra_p0-2)/2.0)-2

    SpazioBiancop0 = CUBOID([blank_space,wall_thickness,3])

    ElementoMuro = FinestrellaCompleta

    window = RIGHT([SpazioBiancop0,RIGHT([COMP([STRUCT, NN(numero_pannelli)])([ElementoMuro, T(1)(lunghezza_pannello_finestra_p0)]),SpazioBiancop0])])
    
    return window

In [9]:
VIEW(getFinestrella(6,1))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x7f33e10beed0> >

![finestra piano 0](../images/workshop08/piano0_finestra_or.jpg)
![finestra piano 0](../images/workshop08/piano0_finestra.png)

* Finestra primo piano

In [10]:
def getFinestraPrimoPiano (lunghezza_pannello_finestra_p1, altezza_pannello_finestra_p1, numero_pannelli_p1):

    Finestra1 = STRUCT([
        INSR(PROD)([QUOTE([0,2,-2,(lunghezza_pannello_finestra_p1-2)/2.0]), QUOTE([0,wall_thickness]), QUOTE([6])]),
        INSR(PROD)([QUOTE([-2,2]), QUOTE([0,wall_thickness]), QUOTE([0.9,-4.5,0.6])])
        ])

    blank_space4 = ((lunghezza_pannello_finestra_p1-2)/2.0)-2
    SpazioBianco4 = CUBOID([blank_space4,wall_thickness,altezza_pannello_finestra_p1])
 
    SpazioBianco2Sopra = CUBOID([lunghezza_pannello_finestra_p1,wall_thickness,((altezza_pannello_finestra_p1-6)*(2.0/3.0))])
    SpazioBianco2Sotto = CUBOID([lunghezza_pannello_finestra_p1,wall_thickness,((altezza_pannello_finestra_p1-6)*(1.0/3.0))])

    Finestra = LEFT([Finestra1,CUBOID([blank_space4,wall_thickness,6])])

    Vfinestra = [[-0.15,0],[-0.15,6-0.6*2],[2.15,6-0.6*2],[2.15,0]]
    FVfinestra = [[0,1],[1,2],[2,3],[3,0]]

    ContornoFinestra1 = STRUCT(MKPOLS((Vfinestra,FVfinestra)))
    ContornoFinestraA = T([1,3,2])([2-0.05,0.6,0.5])(R([2,3])(PI/2)(MULTEXTRUDE(OFFSET([0.1,0.1])(ContornoFinestra1))(0.1)))
    ContornoFinestraB = T([1,3,2])([2-0.1,0.6,0.4])(R([2,3])(PI/2)(MULTEXTRUDE(OFFSET([0.2,0.2])(ContornoFinestra1))(0.2)))
    ContornoFinestraC = T([1,3,2])([2-0.15,0.6,0.3])(R([2,3])(PI/2)(MULTEXTRUDE(OFFSET([0.3,0.3])(ContornoFinestra1))(0.3)))

    Mensola = TEXTURE(texture_finestre)(STRUCT([TOP([CUBOID([3,0.8,0.1]),CUBOID([3,0.5,0.1])])]))

    ContornoFinestra = TEXTURE(texture_finestre)(T(2)(wall_thickness)(STRUCT([ContornoFinestraA,ContornoFinestraB,ContornoFinestraC])))

    FinestraCompleta1 = T(1)(((lunghezza_pannello_finestra_p1-2)/2.0)-2)(STRUCT([Finestra,ContornoFinestra]))

    Vetro1 = getVetro(2,4.8,0.05)
    
    larghezza_persiane = 0.15
    Vlegno = [[0,0.3],[0,6-0.6*2-larghezza_persiane],[2-larghezza_persiane,6-0.6*2-larghezza_persiane],[2-larghezza_persiane,0.3],[(2-larghezza_persiane)/2.0,0.3], [(2-larghezza_persiane)/2.0,6-0.6*2-larghezza_persiane],[0,1.3],[2-larghezza_persiane,1.3],[0,3.5],[2-larghezza_persiane,3.5]]
    FVlegno = [[0,1],[1,2],[2,3],[3,0],[4,5],[6,7],[8,9]]

    Legno1 = STRUCT(MKPOLS((Vlegno,FVlegno)))
    Legno = TEXTURE(texture_persiane)(T([1,2,3])([larghezza_persiane,0.1,larghezza_persiane])((R([2,3])(PI/2)(MULTEXTRUDE(OFFSET([-larghezza_persiane,-larghezza_persiane])(Legno1))(0.2)))))
    Vetro = STRUCT([Vetro1,Legno])

    FinestraCompleta2 = STRUCT([FinestraCompleta1, T([1,2,3])([(lunghezza_pannello_finestra_p1-2)/2.0,wall_thickness,0.6])(Vetro), T([1,2,3])([((lunghezza_pannello_finestra_p1-2)/2.0)-0.5,wall_thickness-0.2,0.6-0.2])(Mensola), T([1,2,3])([((lunghezza_pannello_finestra_p1-2)/2.0)+2.5,wall_thickness-0.2,0.6*2+4.8])(R([1,3])(PI)(Mensola))])

    FinestraCompleta = T(3)(((altezza_pannello_finestra_p1-6)*(1/3.0)))(STRUCT([FinestraCompleta2, T(3)(6)(SpazioBianco2Sopra),T(3)(-((altezza_pannello_finestra_p1-6)*(1/3.0)))(SpazioBianco2Sotto)]))
    
    ElementoMuroPiano1 = FinestraCompleta
    
    MuroPrimoPiano = RIGHT([SpazioBianco4,RIGHT([COMP([STRUCT, NN(numero_pannelli_p1)])([ElementoMuroPiano1, T(1)(lunghezza_pannello_finestra_p1)]),SpazioBianco4])])

    return MuroPrimoPiano

In [11]:
VIEW(getFinestraPrimoPiano(6,8,1))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x7f33e108ba20> >

![finestra piano 1](../images/workshop08/piano1_finestra_or.jpg)
![finestra piano 1](../images/workshop08/piano1_finestra.png)

* Finestra piano 2

In [12]:
def getFinestraSecondoPiano (lunghezza_pannello_finestra_p1, altezza_pannello_finestra_p1, numero_pannelli_p1):

    Finestra1 = STRUCT([
        INSR(PROD)([QUOTE([0,2,-2,(lunghezza_pannello_finestra_p1-2)/2.0]), QUOTE([0,wall_thickness]), QUOTE([3])]),
        INSR(PROD)([QUOTE([-2,2]), QUOTE([0,wall_thickness]), QUOTE([0.6,-1.8,0.6])])
        ])

    blank_space4 = ((lunghezza_pannello_finestra_p1-2)/2.0)-2
    SpazioBianco4 = CUBOID([blank_space4,wall_thickness,altezza_pannello_finestra_p1])

    SpazioBianco2Sopra = CUBOID([lunghezza_pannello_finestra_p1,wall_thickness,((altezza_pannello_finestra_p1-3)*(2.0/3.0))])
    SpazioBianco2Sotto = CUBOID([lunghezza_pannello_finestra_p1,wall_thickness,((altezza_pannello_finestra_p1-3)*(1.0/3.0))])

    Finestra = LEFT([Finestra1,CUBOID([blank_space4,wall_thickness,3])])

    Vfinestra = [[0,0],[0,3-0.6*2],[2,3-0.6*2],[2,0]]
    FVfinestra = [[0,1],[1,2],[2,3],[3,0]]

    ContornoFinestra1 = STRUCT(MKPOLS((Vfinestra,FVfinestra)))
 
    ContornoFinestraC = T([1,3,2])([2-0.2,0.6-0.2,0.1])(R([2,3])(PI/2)(MULTEXTRUDE(OFFSET([0.4,0.4])(ContornoFinestra1))(0.1)))

    ContornoFinestra = TEXTURE(texture_pietra)(T(2)(wall_thickness)(STRUCT([ContornoFinestraC])))

    FinestraCompleta1 = T(1)(((lunghezza_pannello_finestra_p1-2)/2.0)-2)(STRUCT([Finestra,ContornoFinestra]))

    Vetro = getVetro(2,1.8,0.05)

    FinestraCompleta2 = STRUCT([FinestraCompleta1, T([1,2,3])([(lunghezza_pannello_finestra_p1-2)/2.0,wall_thickness,0.6])(Vetro)])

    FinestraCompleta = T(3)(((altezza_pannello_finestra_p1-3)*(1/3.0)))(STRUCT([FinestraCompleta2, T(3)(3)(SpazioBianco2Sopra),T(3)(-((altezza_pannello_finestra_p1-3)*(1/3.0)))(SpazioBianco2Sotto)]))
    
    ElementoMuroPiano1 = FinestraCompleta
    
    window = RIGHT([SpazioBianco4,RIGHT([COMP([STRUCT, NN(numero_pannelli_p1)])([ElementoMuroPiano1, T(1)(lunghezza_pannello_finestra_p1)]),SpazioBianco4])])

    return window

In [13]:
VIEW(getFinestraSecondoPiano(6,4,1))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x7f33e108bbd0> >

![finestra piano 2](../images/workshop08/piano2_finestra_or.jpg)
![finestra piano 2](../images/workshop08/piano2_finestra.png)

* Finestra a forma di arco

In [14]:
# Finestra del secondo piano a forma di arco

def getFinestraArco(lunghezza,spessore,altezza):    
    Parete = CUBOID([spessore,lunghezza,altezza])
    
    diametro_arco = lunghezza*0.70
    Arco = T([1,2,3])([spessore*1.5,lunghezza/2.0,altezza*0.05])(R([1,3])(PI/2)(JOIN([T(2)(-diametro_arco*0.5)(getMapSemiarco(diametro_arco,0)),T(2)(-diametro_arco*0.5)(getMapSemiarco(diametro_arco,spessore*2))])))
    
    Finestra1 = DIFFERENCE([Parete,Arco])
    
    Pilastro = CUBOID([spessore,diametro_arco/8.0,altezza])

    Vetro = T(1)(spessore/2.0)(getVetro(0.05,altezza,lunghezza))
    
    Finestra = STRUCT([Finestra1,T(2)(lunghezza*0.3)(Pilastro),T(2)(lunghezza*0.7-diametro_arco/8.0)(Pilastro),Vetro])
    
    return(Finestra)

In [15]:
VIEW(getFinestraArco(8,0.3,4))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x7f33e108b750> >

![finestra piano 2](../images/workshop08/piano2_finestra_arco.jpg)
![finestra piano 2](../images/workshop08/piano2_finestra_arco.png)

* Porta retro

In [16]:
# Finestra e porta retro

def getPorta (lunghezza_pannello_porta_p1, altezza_pannello_porta_p1):
   
    Porta1 = STRUCT([
        INSR(PROD)([QUOTE([0,2,-2,(lunghezza_pannello_porta_p1-2)/2.0]), QUOTE([0,wall_thickness]), QUOTE([altezza_pannello_porta_p1])]),
        INSR(PROD)([QUOTE([-2,2]), QUOTE([0,wall_thickness]), QUOTE([-4.8,altezza_pannello_porta_p1-4.8])])
        ])
    
    blank_space = ((lunghezza_pannello_porta_p1-2)/2.0)-2
    SpazioBianco = CUBOID([blank_space,wall_thickness,altezza_pannello_porta_p1])
    
    Porta = LEFT([Porta1,SpazioBianco])

    Vporta = [[0,0],[0,6-0.6*2],[2,6-0.6*2],[2,0]]
    FVporta = [[0,1],[1,2],[2,3]]

    ContornoPorta1 = STRUCT(MKPOLS((Vporta,FVporta)))
    ContornoPortaA = T([1,2])([2-0.05,0.5])(R([2,3])(PI/2)(MULTEXTRUDE(OFFSET([0.1,0.1])(ContornoPorta1))(0.1)))
    ContornoPortaB = T([1,2])([2-0.1,0.4])(R([2,3])(PI/2)(MULTEXTRUDE(OFFSET([0.2,0.2])(ContornoPorta1))(0.2)))
    ContornoPortaC = T([1,2])([2-0.15,0.3])(R([2,3])(PI/2)(MULTEXTRUDE(OFFSET([0.3,0.3])(ContornoPorta1))(0.3)))

    Mensola = TEXTURE(texture_finestre)(STRUCT([TOP([CUBOID([3,0.8,0.1]),CUBOID([3,0.5,0.1])])]))

    ContornoPorta = TEXTURE(texture_finestre)(T(2)(wall_thickness)(STRUCT([ContornoPortaA,ContornoPortaB,ContornoPortaC])))

    PortaCompleta1 = T(1)(((lunghezza_pannello_porta_p1-2)/2.0)-2)(STRUCT([Porta,ContornoPorta,T([1,2])([6,wall_thickness])(R([1,2])(-PI)(ContornoPorta))]))

    PortaCompleta = STRUCT([PortaCompleta1, T([1,2,3])([((lunghezza_pannello_porta_p1-2)/2.0)+2.5,wall_thickness-0.2,0.6+4.8])(R([1,3])(PI)(Mensola))])

    return PortaCompleta

In [17]:
VIEW(getPorta(6,8))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x7f33e10496f0> >

![porta piano 1](../images/workshop08/piano1_porta.jpg)
![porta piano 1](../images/workshop08/piano1_porta.png)

* Angolari dell'edificio

In [18]:
# Ritorna 4 angolari che chiudono un rettangolo di dimensione sideAXsideB

def getAngolari(sideA,sideB,altezza):
    
    AngolareMattoni = UNION([(STRUCT([T([1,2,3])([-0.2,-0.2,altezza])(getMattoniLaterali2()),T(2)(-0.2)(R([1,2])(PI/2)(T([1,2,3])([0,0,altezza])(getMattoniLaterali2())))]))])
    Angolari4 = STRUCT([AngolareMattoni,
                    T([1,2])([sideA+blank_space*2,0])(R([1,2])(PI/2)(AngolareMattoni)),
                    T([1,2])([0,sideB+blank_space*2])(R([1,2])(-PI/2)(AngolareMattoni)),
                    T([2,1])([sideB+blank_space*2,sideA+blank_space*2])(R([1,2])(PI)(AngolareMattoni))])
    return TEXTURE(texture_pietraP0)(Angolari4)

In [19]:
VIEW(getAngolari(1,2,2))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x7f33e10497b0> >

![spigoli](../images/workshop08/spigoli.jpg)
![spigoli](../images/workshop08/spigoli.png)

### Struttura piano 0

In [20]:
MuroLaterale = getFinestrella(window_panel_size,side_windows_number)

DueMuriLaterali = STRUCT([T([2,3])([wall_thickness,3])(R([2,3])(PI)(MuroLaterale)), T(2)(front_villa_size+2*blank_space-wall_thickness)(MuroLaterale)])

#MuroPosteriore = T(1)(wall_thickness)(R([1,2])(PI/2)(getFinestrella(window_panel_size,rear_windows_number)))
ElementoMuroPosteriore = T(1)(wall_thickness)(R([1,2])(PI/2)(getFinestrella(window_panel_size,2)))
MuroPosteriore = STRUCT([ElementoMuroPosteriore, T(2)(window_panel_size*4.0)(ElementoMuroPosteriore)])

ElementoMuroFrontale = getFinestrella(window_panel_size,1)

MuroFrontale = T([1,2,3])([side_villa_size*2.0+blank_space*2-wall_thickness,-blank_space*2-(-window_panel_size+8),3])(R([1,3])(PI)(T(1)(side_villa_size)(T(2)(2)(COMP([STRUCT, NN(2)])([R([1,2])(PI/2)(ElementoMuroFrontale), T(2)(front_villa_size-(front_villa_size*1.0/rear_windows_number))])))))

PerimetroPiano0 = STRUCT([DueMuriLaterali, MuroPosteriore, MuroFrontale])

# Base Piano 0
PavimentoP0 = CUBOID([side_villa_size+blank_space*2.0+0.1,front_villa_size+blank_space*2.0+0.1,wall_thickness])

# Tetto Piano 0
SoffittoP0 = TEXTURE(texture_pietra)(TOP([T([1,2,3])([-0.25,-0.25,3])((CUBOID([side_villa_size+0.5+blank_space*2.0,front_villa_size+0.5+blank_space*2.0,0.3]))), T([1,2])([-0.1,-0.1])(CUBOID([side_villa_size+0.2+blank_space*2.0,front_villa_size+0.2+blank_space*2.0,wall_thickness*2]))]))

# Foro per scale tra piano 0 e primo piano
ForoScalePO = COLOR(RED)(T([1,2,3])([window_panel_size,window_panel_size*2.0+blank_space-4,3])(CUBOID([window_panel_size+blank_space*2.0,4,0.3+wall_thickness*2])))
SoffittoP0 = DIFFERENCE([SoffittoP0,ForoScalePO])
SoffittoP0 = TEXTURE(texture_pietra)(DIFFERENCE([SoffittoP0,T(2)((front_villa_size+blank_space*2.0)-(window_panel_size*4.0-blank_space*2.0-wall_thickness))(ForoScalePO)]))

# Muro davanti le scale - piano 0
MuroDavantiScale0 = T([1,2])([side_villa_size+blank_space*2.0-0.2,front_villa_size*1.0/rear_windows_number])((TEXTURE(texture_pietraP0)(CUBOID([0.2,front_villa_size-2*window_panel_size,3]))))

Angolari4 = getAngolari(side_villa_size,front_villa_size,2)

Piano01 = STRUCT([PerimetroPiano0, MuroDavantiScale0, Angolari4])

Piano0 = TOP([PavimentoP0,TOP([Piano01,SoffittoP0])])

In [21]:
VIEW(Piano0)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x7f33e1065210> >

![struttura](../images/workshop08/piano0_struttura_lato1.png)
![struttura](../images/workshop08/piano0_struttura_lato2.png)

In [22]:
VIEW(STRUCT([TOP([lawn,Piano0])]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x7f33e1065330> >

![struttura](../images/workshop08/piano0.png)

### Struttura piano 1

In [23]:
# Piano 1

SpazioBianco4 = CUBOID([blank_space,wall_thickness,first_floor_height])

MuroLateraleP1 = getFinestraPrimoPiano(window_panel_size,first_floor_height,side_windows_number)

DueMuriLateraliP1 = STRUCT([T([2,1])([wall_thickness,side_villa_size+2.0*blank_space])(R([1,2])(PI)(MuroLateraleP1)), T([3,2])([0,front_villa_size+blank_space*2.0-wall_thickness])(MuroLateraleP1)])

#MuroPosterioreP1 = T(1)(wall_thickness)(R([1,2])(-3*PI/2)(getFinestraPrimoPiano(window_panel_size,first_floor_height,rear_windows_number)))
ElementoMuroPosterioreP1 = T(1)(wall_thickness)(R([1,2])(-3*PI/2)(getFinestraPrimoPiano(window_panel_size,first_floor_height,2)))
MuroPosterioreP1 = STRUCT([ElementoMuroPosterioreP1, T(2)(window_panel_size*4.0)(ElementoMuroPosterioreP1)])

ElementoMuroP1 = getFinestraPrimoPiano(window_panel_size,first_floor_height,1)
MuroFrontaleP1 = T([1,2])([side_villa_size+2.0*blank_space-wall_thickness,window_panel_size+2.0*blank_space])(COMP([STRUCT, NN(2)])([R([1,2])(-PI/2)(ElementoMuroP1), T(2)(front_villa_size-window_panel_size)]))

SoffittoP1 = CUBOID([side_villa_size+blank_space*2,front_villa_size+blank_space*2.0,wall_thickness])
ForoPerSoffitto = COLOR(RED)(T([1,2])([wall_thickness,window_panel_size*2.0+blank_space])(CUBOID([side_villa_size-window_panel_size*2.0,front_villa_size-window_panel_size*4.0,wall_thickness])))
SoffittoP1 = DIFFERENCE([SoffittoP1,ForoPerSoffitto])

# Foro per scale tra piano 0 e primo piano
ForoScaleP1 = COLOR(RED)(T([1,2])([window_panel_size,window_panel_size*2.0+blank_space-4])(CUBOID([window_panel_size+blank_space*2.0,4,0.3+wall_thickness*2])))
PavimentoP1 = CUBOID([side_villa_size+blank_space*2,front_villa_size+blank_space*2.0,0.1])
PavimentoP1 = DIFFERENCE([PavimentoP1,ForoScaleP1])
PavimentoP1 = TEXTURE(texture_pavimentoP1)(DIFFERENCE([PavimentoP1,T(2)((front_villa_size+blank_space*2.0)-(window_panel_size*4.0-blank_space*2.0-wall_thickness))(ForoScaleP1)]))


Piano11 = STRUCT([DueMuriLateraliP1, MuroPosterioreP1, MuroFrontaleP1])

Finestra = getFinestraPrimoPiano(6,first_floor_height,1)
Porta = getPorta(6,first_floor_height)
Retro = R([1,2])(-3*PI/2)(STRUCT([Finestra,T(1)(4)(Porta),T(1)(8)(Finestra)]))

Piano1 = STRUCT([TOP([PavimentoP1,TOP([Piano11,SoffittoP1])]),T([1,2,3])([wall_thickness,(window_panel_size+blank_space)*2.0,0.1])(Retro)])

In [24]:
VIEW((Piano1))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x7f33e1065f30> >

![struttura](../images/workshop08/piano1_struttura_lato1.png)
![struttura](../images/workshop08/piano1_struttura_lato2.png)

### Struttura piano 2

In [25]:
# Piano 2

blank_space5 = ((window_panel_size-2)/2.0)-2
SpazioBianco5 = CUBOID([blank_space,wall_thickness,first_floor_height])

MuroLateraleP2 = getFinestraSecondoPiano(window_panel_size,first_floor_height*0.4,side_windows_number-1)

DueMuriLateraliP2 = STRUCT([T([2,1])([wall_thickness,(side_villa_size+2.0*blank_space5)-window_panel_size])(R([1,2])(PI)(MuroLateraleP2)),
                            T([3,2])([0,front_villa_size+blank_space5*2.0-wall_thickness])(MuroLateraleP2)])

ElementoMuroPosterioreP2 = T(1)(wall_thickness)(R([1,2])(-3*PI/2)(getFinestraSecondoPiano(window_panel_size,first_floor_height*0.4,2)))
MuroPosterioreP2 = STRUCT([ElementoMuroPosterioreP2, T(2)(window_panel_size*4.0)(ElementoMuroPosterioreP2)])

ElementoMuroP2 = getFinestraSecondoPiano(window_panel_size,first_floor_height,1)
ElementoMuroP1 = getFinestraPrimoPiano(window_panel_size,first_floor_height*0.6,1)

Tramezzo = T([2,1])([window_panel_size+blank_space5*2.0-wall_thickness,side_villa_size-window_panel_size])(CUBOID([window_panel_size+blank_space5*2,wall_thickness,first_floor_height*0.6+wall_thickness]))
DueTramezzi = STRUCT([Tramezzo,T(2)(side_villa_size-blank_space5*2.0+wall_thickness)(Tramezzo)])

MuroFrontaleP21 = T([1,2])([side_villa_size+2.0*blank_space5-wall_thickness,window_panel_size+2.0*blank_space5])(COMP([STRUCT, NN(2)])([R([1,2])(-PI/2)(ElementoMuroP1), T(2)(front_villa_size-window_panel_size)]))

MuroFrontaleP2 = STRUCT([MuroFrontaleP21,
                         T([1,2])([side_villa_size-window_panel_size,front_villa_size+blank_space5*2.0-wall_thickness])(ElementoMuroP1),
                         T([1,2])([side_villa_size+window_panel_size-blank_space5*6,wall_thickness])(R([1,2])(PI)(ElementoMuroP1))])

Arco = getFinestraArco(front_villa_size-window_panel_size*4.0-blank_space5*2.0,wall_thickness,first_floor_height*0.4)
FinestraArco = T(2)((window_panel_size+blank_space)*2.0)(Arco)

PavimentoP2 = CUBOID([side_villa_size+blank_space*2,front_villa_size+blank_space*2.0,0.1])
ForoPerPavimento = COLOR(RED)(T([1,2])([wall_thickness,window_panel_size*2.0+blank_space])(CUBOID([side_villa_size-window_panel_size*2.0,front_villa_size-window_panel_size*4.0,wall_thickness])))
PavimentoP2 = TEXTURE(texture_pavimentoP2)(DIFFERENCE([PavimentoP2,ForoPerPavimento]))

SoffittoP2_1 = T(3)(first_floor_height*0.4)(CUBOID([side_villa_size-window_panel_size,front_villa_size+blank_space*2.0,wall_thickness]))
SoffittoP2_2 = T([1,2,3])([side_villa_size-window_panel_size,0,first_floor_height*0.6])(CUBOID([window_panel_size+blank_space*2.0,window_panel_size+blank_space*2.0,wall_thickness]))
SoffittoP2_3 = T(2)(front_villa_size-window_panel_size)(SoffittoP2_2)
#SoffittoP2_4 = T([1,3])([side_villa_size-window_panel_size,first_floor_height*0.4])(CUBOID([wall_thickness,front_villa_size+blank_space5*2.0,first_floor_height*0.6-first_floor_height*0.4+wall_thickness]))
SoffittoP2_41 = T([1,3])([side_villa_size-window_panel_size,first_floor_height*0.4])(CUBOID([wall_thickness,window_panel_size+blank_space*2.0,first_floor_height*0.6-first_floor_height*0.4+wall_thickness]))
SoffittoP2_4 = STRUCT([SoffittoP2_41, T(2)(side_villa_size+window_panel_size)(SoffittoP2_41)])
SoffittoP2_5 = T([1,2])([side_villa_size-window_panel_size,window_panel_size+blank_space*2])(CUBOID([wall_thickness,front_villa_size-window_panel_size*2.0-blank_space*2.0,first_floor_height*0.4+wall_thickness]))

SoffittoP2 = STRUCT([SoffittoP2_1,SoffittoP2_2,SoffittoP2_3,SoffittoP2_4,SoffittoP2_5])


Piano2 = STRUCT([PavimentoP2,DueMuriLateraliP2, MuroPosterioreP2, MuroFrontaleP2, SoffittoP2, DueTramezzi, FinestraArco])

In [26]:
VIEW(Piano2)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x7f33e1074c30> >

![struttura](../images/workshop08/piano2_struttura_lato1.png)
![struttura](../images/workshop08/piano2_struttura_lato2.png)

### Struttura completa

In [27]:
texture_beige = 'texture/ivory.jpg'
VIEW(STRUCT([TOP([TOP([lawn,Piano0]),TEXTURE(texture_beige)(Piano1)]),T([1,2,3])
             ([second_floor_X,second_floor_Y,second_floor_height+0.1])(TEXTURE(texture_beige)(Piano2))]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x7f33e1049f30> >

![struttura](../images/workshop08/struttura_lato1.png)
![struttura](../images/workshop08/struttura_lato2.png)

### Scalinate

* Scalinata posteriore

In [28]:
# Scalinata posteriore di 12 gradini

def getScalinoPosteriore(l,altezza,profondita):
    elemento = CUBOID([l,profondita,altezza])
    return elemento

def getScalinataPosteriore(altezza,larghezza,numero_gradini,profondita_scala,lunghezza_cima,altezza_bordo,altezza_cima_dx,altezza_cima_sx):
        
    profondita_gradino = profondita_scala*1.0/numero_gradini
    altezza_scalino = altezza*1.0/numero_gradini
    
    profondita_scala = numero_gradini*profondita_gradino
    scalini = [T(3)(altezza_scalino*i)(getScalinoPosteriore(larghezza*0.8,altezza_scalino,profondita_scala-(profondita_gradino*i))) for i in np.arange(numero_gradini)]
    rampa = [T(3)(altezza_scalino*i)(getScalinoPosteriore(larghezza*0.1,altezza_bordo+altezza_scalino,profondita_scala-(profondita_gradino*i))) for i in np.arange(numero_gradini)]
    
    scalini = STRUCT(scalini)
    rampa = JOIN(STRUCT(rampa))
    
    cima1 = CUBOID([larghezza*0.8,lunghezza_cima,altezza])
    cima_bordo_sx = CUBOID([larghezza*0.1,lunghezza_cima,altezza+altezza_cima_sx])
    cima_bordo_dx = CUBOID([larghezza*0.1,lunghezza_cima,altezza+altezza_cima_dx])
    
    cima = STRUCT([RIGHT([cima_bordo_sx,RIGHT([cima1,cima_bordo_dx])])])
    
    scalinata1 = STRUCT([RIGHT([rampa,RIGHT([scalini,rampa])])])
    scalinata = T(2)(-larghezza/2.0)(R([1,2])(PI/2)(STRUCT([UP([cima,scalinata1])])))
    return scalinata

Scalinata_posteriore = getScalinataPosteriore(floor_height_0,window_panel_size*2.0-blank_space*2.0,13,6.5,2,1,1,1)

In [29]:
VIEW(getScalinataPosteriore(20,10,20,30,5,1,1,1))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x7f33e1080b10> >

![scalinata posteriore](../images/workshop08/scalinata_lato1.png)

In [30]:
# Scalinata frontale di 6 gradini# Scalin 

def Scalino(r,altezza):
    elemento = JOIN([T(2)(-r*0.5)(getMapSemiarco(r,0)),T(2)(-r*0.5)(getMapSemiarco(r,altezza))])
    
    return elemento

def getScalinata(r,altezza_totale):
    
    altezza_scalino = altezza_totale/6.0
    
    scalino_base = Scalino(r,altezza_scalino)
    
    Scalinata1 = T(3)(altezza_scalino*5.0)(STRUCT([
                        scalino_base,
                        T(3)(-altezza_scalino)(S([1,2])([1.15,1.15])(scalino_base)),
                        T(3)(-2*altezza_scalino)(S([1,2])([1.3,1.3])(scalino_base)),
                        T(3)(-3*altezza_scalino)(S([1,2])([1.45,1.45])(scalino_base)),
                        T(3)(-4*altezza_scalino)(S([1,2])([1.6,1.6])(scalino_base)),
                        T(3)(-5*altezza_scalino)(S([1,2])([1.75,1.75])(scalino_base)),
                        ]))

    Scalinata = TEXTURE(texture_pietra)(Scalinata1)
    
    return Scalinata

Scalinata = getScalinata(front_staircase_radius,front_staircase_height)

In [31]:
VIEW(getScalinata(8,2))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x7f33e10807e0> >

![scalinata anteriore](../images/workshop08/scalinata_lato2.png)

In [32]:
VIEW(STRUCT([TOP([TOP([lawn,Piano0]),Piano1]),
             T([1,2,3])([second_floor_X,second_floor_Y,second_floor_height+0.1])(Piano2),
             T([1,2,3])([facet_Y+facet_thickness,lawn_size_y/2.0,floor_height_0-front_staircase_height])(Scalinata),
             T([1,2])([second_floor_X,lawn_size_y/2.0])(Scalinata_posteriore)]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x7f33e1080a20> >

![scalinata anteriore](../images/workshop08/facciata_anteriore.png)
![scalinata posteriore](../images/workshop08/facciata_posteriore.png)

### Facciata anteriore

In [33]:
def getColonneMattoncini(spessore,larghezza,distanza_tra_colonne,altezza_colonna,numero_mattoncini):
    
    altezza_mattoncino = (altezza_colonna*1.0/numero_mattoncini)-0.15

    Mattoncino = CUBOID([spessore,larghezza,altezza_mattoncino])
    
    BaseColonnaMattoncini = CUBOID([spessore-0.1,larghezza,altezza_colonna])
    
    ColonnaMattoncini1 = COMP([STRUCT, NN(numero_mattoncini)])([Mattoncino, T(3)(altezza_mattoncino+0.15)])

    ColonnaMattoncini = UNION([STRUCT([BaseColonnaMattoncini,ColonnaMattoncini1])])

    ColonnaMattoncini4 = COMP([STRUCT, NN(4)])([ColonnaMattoncini, T(2)(distanza_tra_colonne)])

    ColonneMattonciniFinite = TEXTURE(texture_pietra)(STRUCT([ColonnaMattoncini,T(2)(larghezza+0.16)(ColonnaMattoncini4),T(2)(distanza_tra_colonne*3.0+larghezza*2.0+0.16*2)(ColonnaMattoncini)]))
    
    return ColonneMattonciniFinite

In [34]:
VIEW(getColonneMattoncini(3,1.6,5.8,8,22))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x7f33e1080fc0> >

![mattoncini](../images/workshop08/mattoncini.png)

In [35]:
def getColonna(w,h,s):
    basis = CUBOID([w, w, w/ 6.0])
    trunk = CYLINDER([w / 2.0*0.85, h-(2.0*w/6.0)])(24)
    capitel = CUBOID([w, w*1.5+s, w/6.0])
    return TOP([TOP([basis, trunk]), capitel])


def getColonnato(dm,h,numero_colonne,spazio):
    colonnato = STRUCT(NN(numero_colonne)([getColonna(dm,h,spazio),T(2)(spazio+dm)]))
    return TEXTURE(texture_pietra)(colonnato)

In [36]:
VIEW(getColonnato(0.4,1.2,6,0.6))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x7f33e10451e0> >

![colonnato](../images/workshop08/colonnato.png)

In [37]:
# Arco di mattoni

def ArchSurface(rr,w,h):
    x1 = lambda k: rr*cos(k[0])
    y1 = lambda k: rr*sin(k[0])
    x2 = lambda k: (rr-w)*cos(k[0])
    y2 = lambda k: (rr-w)*sin(k[0])
    z = lambda k: h
    
    Circle0 = BEZIER(S1)([CONS([x1,y1,z])])
    
    Circle1 = BEZIER(S1)([CONS([x2,y2,z])])
    
    return BEZIER(S2)([Circle0, Circle1])

def getArco1(length,w,depth):
    def Arch1(angle):
        
        radius=(length/2)/cos(angle/2)
        
        domain2D=PROD([T(1)(angle/2)(INTERVALS(PI-angle)(13)),QUOTE([1])])
        
        domain3D=PROD([domain2D, QUOTE([1])])
        
        ArchSurf2D_0=ArchSurface(radius,w,0)
        
        ArchSurf2D_1=ArchSurface(radius,w,depth)
        
        SolidMap=BEZIER(S3)([ArchSurf2D_0, ArchSurf2D_1])
        
        return MAP(SolidMap)(domain3D)
    
    return Arch1

def getArco(lunghezza,altezza,profondita):
    
    Arco = R([1,2])(PI/2)(R([2,3])(-3*PI/2)(getArco1(lunghezza, altezza, profondita)(0.001*PI)))
    
    return Arco

def getArchi(lunghezza,altezza,profondita,numero_archi,spazio_tra_archi):
    
    Archi = T(2)(lunghezza/2.0)(STRUCT(NN(numero_archi)([getArco(lunghezza,altezza,profondita),T(2)(spazio_tra_archi+lunghezza)])))
    
    return TEXTURE(texture_pietra)(Archi)

In [38]:
VIEW(getArchi(10,2,1.6,3,-2))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x7f33e10587e0> >

![archi](../images/workshop08/archi.png)

In [39]:
def getCornice(numeroElementiPannello,spessore,larghezza,altezza_base,altezza_decorazione):
    
    larghezza_pannello = (larghezza*1.0/numeroElementiPannello)*0.7
    
    larghezza_decorazione = ((larghezza*1.0/numeroElementiPannello)*0.3)/4.0

    ElementoPannello1 = COMP([STRUCT, NN(4)])([CUBOID([spessore,larghezza_decorazione,altezza_decorazione]), T(2)(larghezza_decorazione*1.2)])
    
    BaseElementoPannello = CUBOID([spessore-0.1,larghezza_decorazione*1.2*4-larghezza_decorazione*0.2,altezza_decorazione])

    ElementoPannello = STRUCT ([ElementoPannello1,BaseElementoPannello])

    ElementoPannello3 = CUBOID([spessore-0.1,larghezza_pannello,altezza_decorazione]) # cubo tra elementi

    ElementoPannello4 = STRUCT([ElementoPannello,
                        T(2)(larghezza_decorazione*1.2*4-larghezza_decorazione*0.2)(ElementoPannello3),
                        T(3)(altezza_decorazione)(CUBOID([spessore,larghezza_pannello+larghezza_decorazione*1.2*4-larghezza_decorazione*0.2,0.2])),
                        CUBOID([spessore,larghezza_pannello+larghezza_decorazione*1.2*4-larghezza_decorazione*0.2,0.2]),
                        T(3)(-altezza_base)(CUBOID([spessore-0.1,larghezza_pannello+larghezza_decorazione*1.2*4-larghezza_decorazione*0.2,altezza_base]))
                        ])

    ElementoUltimoPannello = STRUCT([ElementoPannello,
                            T(3)(altezza_decorazione)(CUBOID([spessore,larghezza_decorazione*1.2*4-larghezza_decorazione*0.2,0.2])),
                            CUBOID([spessore,larghezza_decorazione*1.2*4-larghezza_decorazione*0.2,0.2]),
                            T(3)(-altezza_base)(CUBOID([spessore-0.1,larghezza_decorazione*1.2*4-larghezza_decorazione*0.2,altezza_base]))
                        ])

    ElementoPannelloCompleto1 = COMP([STRUCT, NN(numeroElementiPannello)])([ElementoPannello4, T(2)(larghezza_pannello+larghezza_decorazione*1.2*4-larghezza_decorazione*0.2)])
    ElementoPannelloCompleto = T(3)(altezza_base)(STRUCT([ElementoPannelloCompleto1, T(2)(((larghezza_decorazione*1.2*4-larghezza_decorazione*0.2)+larghezza_pannello)*numeroElementiPannello)(ElementoUltimoPannello)]))
    
    return TEXTURE(texture_pietra)(UNION([ElementoPannelloCompleto]))

In [40]:
VIEW(getCornice(8,1.6,10,1,1))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x7f33e1058930> >

![cornice](../images/workshop08/cornice.png)

In [41]:
# Trinagolo su archi

def getTriangolo(lunghezza,altezza,spessore):
    punti = [[0,0],[0,lunghezza],[altezza,lunghezza/2.0]]
    P = AA(MK)(punti)
    S = AA(JOIN)([P[0],P[1],P[2]])
    H = JOIN(S)
    Triangolo = R([2,3])(PI/2)(MULTEXTRUDE(H)(1.6))

    V = [[0,0],[0,0.5],[0.5,0.5],[0.5,1],[1,1],[1,0.5],[1.5,0.5],[1.5,0]]
    FV = [[0,1,3,7],[2,3,4,7],[4,5,6,7]]

    A = STRUCT(MKPOLS((V,FV)))

    V1 = [[0,0],[0,lunghezza],[altezza,lunghezza/2.0]]
    FV1 = [[0,1],[1,2],[0,2]]

    ContornoTriangolo1 = STRUCT(MKPOLS((V1,FV1)))
    ContornoTriangolo = R([2,3])(PI/2)(MULTEXTRUDE(OFFSET([1,0])(ContornoTriangolo1))(2))
    ContornoTriangoloGrande = R([2,3])(PI/2)(MULTEXTRUDE(OFFSET([0.2,0])(ContornoTriangolo1))(2.2))
    ContornoTriangoloDietro = R([2,3])(PI/2)(MULTEXTRUDE(OFFSET([2,1])(ContornoTriangolo1))(spessore))

    TriangoloCompleto = T([1,2])([spessore,0.5])(R([2,3])(-PI/2)(R([1,2])(-PI/2)(STRUCT([Triangolo, 
                                                                       T(2)(0.8)(ContornoTriangolo),
                                                                       T([2,1])([1.2,0.5])(ContornoTriangoloGrande),
                                                                       T(3)(-0.5)(ContornoTriangoloDietro)
                                                                      ]))))
    
    return TEXTURE(texture_pareti)(UNION([TriangoloCompleto]))

In [42]:
VIEW(getTriangolo(26,5,1.6))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x7f33e106b330> >

![frontone](../images/workshop08/frontone1.png)
![frontone](../images/workshop08/frontone2.png)

In [43]:
ColonneMattoncini = TEXTURE(texture_pietra)(getColonneMattoncini(facet_thickness,2,7.9,12,14))

Archi = TEXTURE(texture_pietra)(T([2,3])([2+0.16,7.5])(getArchi(8.3+1.6,2,facet_thickness,3,-2)))

Cornicione = TEXTURE(texture_pietra)(T(3)(11)(getCornice(14,facet_thickness,28.1,1.6,1.5)))

Triangolo = T(3)(11+1.6+1.5)(getTriangolo(29.05,7,facet_thickness))

Colonnato = T([2,1,3])([4.24,facet_thickness/2.0,0.2])(getColonnato(0.5,1.5,9,0.15))
Colonnato = STRUCT([Colonnato,T(2)(7.9*2.0)(Colonnato)])

Facciata = STRUCT([ColonneMattoncini, Archi, Cornicione, Triangolo, Colonnato])

In [44]:
VIEW(Facciata)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x7f33e106b300> >

![facciata](../images/workshop08/facciata_frontale.png)

In [45]:
VIEW(STRUCT([TOP([TOP([lawn,Piano0]),TEXTURE(texture_beige)(Piano1)]),
             T([1,2,3])([second_floor_X,second_floor_Y,second_floor_height+0.1])(TEXTURE(texture_beige)(Piano2)),
             T([1,2,3])([facet_Y+facet_thickness,lawn_size_y/2.0,floor_height_0-front_staircase_height+0.1])(Scalinata),
             T([1,2])([second_floor_X,lawn_size_y/2.0])(Scalinata_posteriore),
             T([1,2,3])([facet_Y,facet_X,facet_Z])(Facciata)]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x7f33e106b060> >

![facciata](../images/workshop08/facciata_frontale1.png)
![facciata](../images/workshop08/facciata_frontale2.png)
![facciata](../images/workshop08/facciata_frontale3.png)

### Tetto

In [46]:
def getTetto1(side,altezza):
    V1 = [[0,0,-altezza],[0,side,-altezza],[side,0,-altezza],[side,side,-altezza],[side/2.0,side/2.0]]
    FV1 = [[0,2,4],[2,3,4]]

    Tetto1 = STRUCT(MKPOLS((V1,FV1)))
    
    Tetto = T(3)(altezza)(STRUCT([Tetto1,R([1,2])(PI)(T([1,2])([-side,-side])(Tetto1))]))
    
    return TEXTURE(texture_tetto)(Tetto)

def getTetto2(sideA,sideB,altezza):
    
    V1 = [[0,0,-altezza],[0,sideA,-altezza],[sideB,0,-altezza],[sideB,sideA,-altezza],[sideB*0.35,sideA/2.0],[sideB*0.65,sideA/2.0]]

    FVa = [[2,3,5]]
    FVb = [[1,3],[4,5]]
   
    TettoA = STRUCT(MKPOLS((V1,FVa)))
    TettoB = JOIN(STRUCT(MKPOLS((V1,FVb))))
    
    Tetto1 = STRUCT([TettoA,TettoB])

    Tetto = T(3)(altezza)(STRUCT([Tetto1,R([1,2])(PI)(T([1,2])([-sideB,-sideA])(Tetto1))]))
    
    return TEXTURE(texture_tetto)(Tetto)

def getTetto3(lunghezza,altezza,spessore):
    
    V1 = [[0,0],[0,lunghezza],[altezza,lunghezza/2.0]]
    
    FV1 = [[1,2],[0,2]]
    
    TriangoloPerTetto = STRUCT(MKPOLS((V1,FV1)))
    
    Tetto = R([1,3])(PI/2)(MULTEXTRUDE(TriangoloPerTetto)(spessore))
    
    return TEXTURE(texture_tetto)(Tetto)

In [49]:
side = window_panel_size + blank_space*2.0
TettoQuadrato = JOIN(getTetto1(side, roofs_height))

TettiQuadrati = TEXTURE(texture_tetto)(COMP([STRUCT, NN(2)])([TettoQuadrato,T(2)(front_villa_size-window_panel_size)]))

TettoCentrale = TEXTURE(texture_tetto)(JOIN(R([1,2])(PI/2)(getTetto2(side_villa_size-window_panel_size,front_villa_size+blank_space*2.0,roofs_height*4))))

TettoTriangolo = JOIN(T([1,2])([side,side+0.5])(getTetto3(29.05,7,side_villa_size+blank_space*2.0-(side_villa_size-window_panel_size)/2.0)))

TettoDietroTriangolo = TEXTURE(texture_tetto)(T([2,3])([window_panel_size+blank_space*2.0,-(first_floor_height*0.4+wall_thickness)])(CUBOID([wall_thickness*2.0,front_villa_size-window_panel_size*2.0-blank_space*2.0,first_floor_height*0.4+wall_thickness])))


da_sottrarre1 = COLOR(GREEN)(JOIN(TettoCentrale))

TettoTriangolo = TEXTURE(texture_tetto)(DIFFERENCE([TettoTriangolo,da_sottrarre1]))

Tetto1 = STRUCT([T(3)(first_floor_height*0.6-first_floor_height*0.4)(TettiQuadrati),
                T(1)(0)(TettoCentrale),
                T(3)(-first_floor_height*0.4+1)(TettoTriangolo)])

Tetto = STRUCT([Tetto1,TettoDietroTriangolo])

In [50]:
VIEW(Tetto)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x7f33e0e62690> >

![tetto](../images/workshop08/tetto1.png)
![tetto](../images/workshop08/tetto2.png)

In [51]:
VIEW(STRUCT([TOP([TOP([lawn,Piano0]),Piano1]),
             T([1,2,3])([second_floor_X,second_floor_Y,second_floor_height+0.1])(Piano2),
             T([1,2,3])([roof_X,roof_Y,roof_height+0.1])(Tetto),
             T([1,2,3])([facet_Y+facet_thickness,lawn_size_y/2.0,floor_height_0-front_staircase_height+0.1])(Scalinata),
             T([1,2])([second_floor_X,lawn_size_y/2.0])(Scalinata_posteriore),
             T([1,2,3])([facet_Y,facet_X,facet_Z])(Facciata)]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x7f33e0e62ae0> >

### Villa completa

In [53]:
VIEW(STRUCT([TOP([TOP([lawn,Piano0]),TEXTURE(texture_beige)(Piano1)]),
             T([1,2,3])([second_floor_X,second_floor_Y,second_floor_height+0.1])(TEXTURE(texture_beige)(Piano2)),
             T([1,2,3])([roof_X,roof_Y,roof_height+0.1])(Tetto),
             T([1,2,3])([facet_Y+facet_thickness,lawn_size_y/2.0,floor_height_0-front_staircase_height+0.1])(Scalinata),
             T([1,2])([second_floor_X,lawn_size_y/2.0])(Scalinata_posteriore),
             T([1,2,3])([facet_Y,facet_X,facet_Z])(Facciata)]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x7f33e106b390> >

![villa](../images/workshop08/villa1.png)
![villa](../images/workshop08/villa2.png)
![villa](../images/workshop08/villa3.png)
![villa](../images/workshop08/villa4.png)

<table>
    <tbody>
        <tr>
            <td><img src="../images/workshop08/VillaPisani_1.jpg"></td>
            <td><img src="../images/workshop08/VillaPisani_1.png"></td>
        </tr>
        <tr>
            <td><img src="../images/workshop08/VillaPisani_2.jpg"></td>
            <td><img src="../images/workshop08/VillaPisani_2.png"></td>
        </tr>
    </tbody>
</table>

### Realizzazione interni

In [54]:
# Muri interni
def getParetiInterne(side_villa_size,front_villa_size,altezza):
    PareteP2_4 = CUBOID([wall_thickness,front_villa_size+blank_space5*2.0,second_floor_height+first_floor_height*0.4+wall_thickness])
    Parete2 = T(2)([window_panel_size+blank_space5*2.0-wall_thickness])(CUBOID([window_panel_size+blank_space5*2,wall_thickness,altezza+wall_thickness]))
    DueParete2 = STRUCT([Parete2,T(2)(side_villa_size-blank_space5*2.0+wall_thickness)(Parete2)])
    
    # riferimento per creare varchi per le porte
    foroPerPorta = CUBOID([wall_thickness,3,6])
    
    # porte primo piano
    PareteP2_4 = DIFFERENCE([PareteP2_4,T([2,3])([-1.5+(front_villa_size+blank_space*2.0)/2.0,floor_height_0])(foroPerPorta)])
    DueParete2 = DIFFERENCE([DueParete2,T([1,2,3])([window_panel_size/2.0,window_panel_size+blank_space*2.0,floor_height_0])(R([1,2])(-PI/2)(foroPerPorta))])
    DueParete2 = DIFFERENCE([DueParete2,T([1,2,3])([window_panel_size/2.0,front_villa_size-(window_panel_size-wall_thickness),floor_height_0])(R([1,2])(-PI/2)(foroPerPorta))])
    PareteP2_4 = DIFFERENCE([PareteP2_4,T([2,3])([-1.5+window_panel_size/2.0+blank_space,floor_height_0])(foroPerPorta)])
    PareteP2_4 = DIFFERENCE([PareteP2_4,T([2,3])([front_villa_size+1.5-window_panel_size/2.0-blank_space,floor_height_0])(foroPerPorta)])
    
    PareteD = T([1,2])([-window_panel_size*3.0,window_panel_size*2.0+blank_space-wall_thickness])(CUBOID([window_panel_size,wall_thickness,second_floor_height+first_floor_height*0.4+wall_thickness]))
    DuePareteD = STRUCT([PareteD,T(2)(window_panel_size*2.0+wall_thickness)(PareteD)])
    
    PareteP2_5 = CUBOID([wall_thickness,front_villa_size+blank_space*2.0,second_floor_height+first_floor_height*0.4+wall_thickness])
    PareteP2_5 = DIFFERENCE([PareteP2_5,T([2,3])([-1.5+(front_villa_size+blank_space*2.0)/2.0,floor_height_0])(foroPerPorta)])
    PareteP2_5 = DIFFERENCE([PareteP2_5,T([2,3])([-1.5+window_panel_size/2.0+blank_space,floor_height_0])(foroPerPorta)])
    PareteP2_5 = DIFFERENCE([PareteP2_5,T([2,3])([front_villa_size+1.5-window_panel_size/2.0-blank_space,floor_height_0])(foroPerPorta)])
    
    # muretto vicino scale secondo piano
    MuroVicinoScale1 = T([1,2,3])([-window_panel_size,window_panel_size*2.0+blank_space,second_floor_height])(CUBOID([wall_thickness,window_panel_size*2.0,3]))
    MuroVicinoScale21 = T([1,2,3])([-window_panel_size*2.0+4.0,window_panel_size*2.0+blank_space,second_floor_height])(CUBOID([window_panel_size-4.0+wall_thickness,wall_thickness,3]))
    MuroVicinoScale22 = T([1,2,3])([-window_panel_size*2.0+4.0,(front_villa_size+blank_space*2.0)-(window_panel_size*2.0+blank_space),second_floor_height])(CUBOID([window_panel_size-4.0+wall_thickness,wall_thickness,3]))
    MuroVicinoScale = STRUCT([MuroVicinoScale1,MuroVicinoScale21,MuroVicinoScale22])
    
    # porta principale primo piano
    Porta1 = T([2,3])([1.5+front_villa_size/2.0+blank_space*2.0,floor_height_0])(R([1,2])(-PI/2)(getPorta(6,6)))
    Porta2 = T([1,2,3])([-window_panel_size*2.0,1.5+front_villa_size/2.0+blank_space*2.0,floor_height_0])(R([1,2])(-PI/2)(getPorta(6,6)))
    Porta = STRUCT([Porta1,Porta2])
    
    # porte secondo piano
    PareteP2_4 = DIFFERENCE([PareteP2_4,T([2,3])([-1.5+window_panel_size/2.0+blank_space,second_floor_height])(foroPerPorta)])
    PareteP2_4 = DIFFERENCE([PareteP2_4,T([2,3])([front_villa_size+1.5-window_panel_size/2.0-blank_space,second_floor_height])(foroPerPorta)])
    PareteP2_5 = DIFFERENCE([PareteP2_5,T([2,3])([-1.5+window_panel_size/2.0+blank_space,second_floor_height])(foroPerPorta)])
    PareteP2_5 = DIFFERENCE([PareteP2_5,T([2,3])([front_villa_size+1.5-window_panel_size/2.0-blank_space,second_floor_height])(foroPerPorta)])
    
    # scalinate tra primo e secondo piano
    Scalinata1 = T([1,2,3])([-window_panel_size*2,2+window_panel_size*2.0+blank_space,floor_height_0])(R([1,2])(PI)(getScalinataPosteriore(first_floor_height+wall_thickness+0.1,4,25,8.75,4,3,0,3)))
    Scalinata2 = T([1,2,3])([-window_panel_size*2.0,-2+(front_villa_size+blank_space*2.0+wall_thickness)-(window_panel_size*2+blank_space),floor_height_0])(R([1,2])(PI)(getScalinataPosteriore(first_floor_height+wall_thickness+0.1,4,25,8.75,4,3,3,0)))
    
    # scalinate tra piano 0 e primo piano
    Scalinata01 = T([1,2,3])([-window_panel_size+4,2+window_panel_size*2.0+blank_space-4,-wall_thickness])(getScalinataPosteriore(floor_height_0+wall_thickness+0.1,4,15,4,4,3,0,0))
    Scalinata02 = T([1,2,3])([-window_panel_size+4,-2+(front_villa_size+blank_space*2.0+wall_thickness+4)-(window_panel_size*2+blank_space),-wall_thickness])(getScalinataPosteriore(floor_height_0+wall_thickness+0.1,4,15,4,4,3,0,0))
    
    Scale = STRUCT([Scalinata1,Scalinata2,Scalinata01,Scalinata02])
    
    PareteP2_4 = STRUCT([PareteP2_4,T(1)(-window_panel_size*2.0)(PareteP2_5)])
    
    # costruzione interni della Villa
    Interni = T(3)(-0.1)(STRUCT([PareteP2_4,DueParete2,DuePareteD,Porta,Scale,MuroVicinoScale]))

    return TEXTURE(texture_pareti)(Interni)

In [55]:
VIEW(getParetiInterne(side_villa_size,front_villa_size,walls_height))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x7f33e0e62c00> >

![interni](../images/workshop08/interni1.png)
![interni](../images/workshop08/interni2.png)

In [56]:
VIEW(STRUCT([TOP([Prato,Piano0]),
             T([1,2])([second_floor_X+side_villa_size-window_panel_size,
                       second_floor_Y])(getParetiInterne(side_villa_size,
                                                             front_villa_size,walls_height))]))

NameError: name 'Prato' is not defined

![interni](../images/workshop08/interni3.png)

### Villa completa con interni

In [58]:
VIEW(STRUCT([TOP([TOP([lawn,Piano0]),TEXTURE(texture_beige)(Piano1)]),
             T([1,2,3])([second_floor_X,second_floor_Y,second_floor_height+0.1])(TEXTURE(texture_beige)(Piano2)),
             T([1,2,3])([roof_X,roof_Y,roof_height+0.1])(Tetto),
             T([1,2,3])([facet_Y+facet_thickness,lawn_size_y/2.0,floor_height_0-front_staircase_height+0.1])(Scalinata),
             T([1,2])([second_floor_X,lawn_size_y/2.0])(Scalinata_posteriore),
             T([1,2,3])([facet_Y,facet_X,facet_Z])(Facciata),
             T([1,2])([second_floor_X+side_villa_size-window_panel_size,second_floor_Y])(getParetiInterne(side_villa_size,front_villa_size,walls_height))]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x7f33e0d90b70> >

![villa completa](../images/workshop08/completa.png)
![villa completa](../images/workshop08/interni4.png)